In [1]:
#Import Libraries
import subprocess
import re
import pandas as pd
import os
import collections
import scipy.stats
import numpy as np
import scipy
import csv
import math
import time
import seaborn as sns
import statsmodels.api as sm
import joblib
import random
import extra_funcs_TO

from functools import reduce
from tqdm import tqdm
from matplotlib import pyplot as plt
%matplotlib inline
from pandas.tools.plotting import table
from scipy import interp
from sklearn import svm
from sklearn.svm import SVC
from sklearn.datasets import make_blobs, make_classification


from sklearn.feature_selection import SelectKBest, chi2, VarianceThreshold
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, GridSearchCV, LeaveOneOut, train_test_split, StratifiedShuffleSplit, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, auc, roc_curve, precision_recall_curve, f1_score, classification_report, make_scorer, balanced_accuracy_score, roc_auc_score, matthews_corrcoef
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.multiclass import unique_labels
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from mpl_toolkits.mplot3d import Axes3D

In [2]:
dset=pd.read_csv ("clintab_GMQL/gencode.gene.info.v22.tsv", delimiter='\t')
gruppo=dset.groupby(['gene_type'])
proteingroup=gruppo.get_group('protein_coding')
lista1=set(proteingroup['gene_id'])
#len(proteingroup)

In [3]:
dset_rna_res=pd.read_csv("clintab_GMQL/gene_exprs/res/res_new.csv", delimiter='\t')
dset_rna_senl=pd.read_csv("clintab_GMQL/gene_exprs/sl_32/sl_new.csv", delimiter='\t')
dset_rna_sens=pd.read_csv("clintab_GMQL/gene_exprs/ss_32/ss_new.csv", delimiter='\t')

In [4]:
dset_res = dset_rna_res[dset_rna_res.ensemble_id.isin(lista1)]
protein_coding_res=pd.DataFrame(dset_res)
protein_coding_res['genes'] = protein_coding_res['ensemble_id']+'-'+ protein_coding_res['gene_symbol']
#protein_coding_res.head()

In [5]:
dset_senl = dset_rna_senl[dset_rna_senl.ensemble_id.isin(lista1)]
protein_coding_senl=pd.DataFrame(dset_senl)
protein_coding_senl['genes'] = protein_coding_senl['ensemble_id']+'-'+ protein_coding_senl['gene_symbol']

In [6]:
dset_sens = dset_rna_sens[dset_rna_sens.ensemble_id.isin(lista1)]
protein_coding_sens=pd.DataFrame(dset_sens)
protein_coding_sens['genes'] = protein_coding_sens['ensemble_id']+'-'+ protein_coding_sens['gene_symbol']

In [7]:
resdf=protein_coding_res.pivot(index="patient", columns="genes", values="fpkm")
genes = resdf.columns.tolist()
#resdf.head()
resdf.shape

(60, 19814)

In [8]:
senldf=protein_coding_senl.pivot(index="patient", columns="genes", values="fpkm")
senldf = senldf[genes]
#senldf.head()
senldf.shape

(34, 19814)

In [9]:
sensdf=protein_coding_sens.pivot(index="patient", columns="genes", values="fpkm")
sensdf = sensdf[genes]
#sensdf.head()
sensdf.shape

(107, 19814)

In [69]:
resdf_median = resdf.median(0)
senldf_median = senldf.median(0)
sensdf_median = sensdf.median(0)

In [70]:
resdf_median.shape

(19814,)

In [71]:
print(len(resdf_median), len(senldf_median), len(sensdf_median))

19814 19814 19814


In [72]:
pvalues = []
for gene in tqdm(genes):
    try:
        gene_pvalue = scipy.stats.mannwhitneyu(senldf[gene],sensdf[gene]).pvalue
    except Exception:
        gene_pvalue = 1
    pvalues.append(gene_pvalue)
       
pvalues = pd.DataFrame({
    "gene": genes,
    #"resistant_median": resdf_median.values,
    "sensitive_long_median": senldf_median.values,
    "sensitive_short_median": sensdf_median.values,
    "mannwhiteney_pvalue": pvalues
})

pvalues.head()

100%|██████████| 19814/19814 [00:07<00:00, 2639.39it/s]


,gene,sensitive_long_median,sensitive_short_median,mannwhiteney_pvalue
0,ENSG00000000003.13-TSPAN6,18.905782,15.924936,0.044214
1,ENSG00000000005.5-TNMD,0.074047,0.067985,0.330315
2,ENSG00000000419.11-DPM1,41.232656,43.144585,0.475980
3,ENSG00000000457.12-SCYL3,1.890405,2.014650,0.394546
4,ENSG00000000460.15-C1orf112,1.703900,1.640842,0.470225


In [73]:
threshold = 0.0006
min_median_sum = 0

significant_genes = pvalues[(pvalues.mannwhiteney_pvalue < threshold)].copy()# & 
                           # (pvalues.sensitive_short_median + pvalues.sensitive_long_median > min_median_sum)].copy()
#significant_genes["median_difference"] = (significant_genes.resistant_median - significant_genes.sensitive_short_median).abs()
significant_genes = significant_genes.sort_values("mannwhiteney_pvalue", ascending=True)
print(significant_genes.shape[0])


41


In [74]:
names=[]

for x in ["-".join(x.strip().split("-")[1:])  for x in (significant_genes['gene'])]:
    names.append(x)   
    
significant_genes['gene_symbol']=names
significant_genes=pd.DataFrame(significant_genes)
#significant_genes.to_csv('giada/sl_ss_6.csv', sep=',', header=True, index=False)
#significant_genes.to_csv('significant_genes_res_ss.csv', sep=',', header=True, index=False)
#significant_genes.to_csv('significant_genes_sl_ss.csv', sep=',', header=True, index=False)

In [34]:
common_genes_pc=pd.read_csv('common_genes/common_genes_total_us.csv', sep=';')

In [46]:
genes=(common_genes_pc['ensemble_id_us'])
genes=genes.to_frame()

In [47]:
column=genes['ensemble_id_us']
column.head()

0    ENSG00000137393.9-RNF144B
1     ENSG00000134504.11-KCTD1
2       ENSG00000137364.4-TPMT
3    ENSG00000090520.9-DNAJB11
4     ENSG00000177383.4-MAGEF1
Name: ensemble_id_us, dtype: object

In [55]:
res_selected=resdf[column]
res_selected["label"]=1
res_selected.shape

/home/lalli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(60, 123)

In [62]:
senl_selected=senldf[column]
senl_selected["label"]=1
senl_selected.shape

/home/lalli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(34, 123)

In [63]:
sens_selected=sensdf[column]
sens_selected["label"]=0
sens_selected.shape

/home/lalli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(107, 123)

In [64]:
#concatenated=pd.concat([res_selected, senl_selected])
#concatenated=pd.concat([res_selected, sens_selected])
concatenated=pd.concat([senl_selected, sens_selected])

In [65]:
X=concatenated[concatenated.columns[:-1]].values
y=concatenated[concatenated.columns[-1]].values

In [ ]:
def my_PCA(data, features, labels, i, j):
    #x = data.loc[:, features].values
    x = StandardScaler().fit_transform(data)
    
    pca= PCA(n_components=3)
    p_comp=pca.fit_transform(x)
    #p_comp = TSNE(n_components=2, perplexity=70, n_iter=2500).fit_transform(x)
    p_comp=p_comp[:, [i,j]]

    principalDf = pd.DataFrame(data = p_comp
             , columns = ['principal component '+str(i), 'principal component '+str(j)])
    finalDf= pd.concat([principalDf, pd.Series(labels)], axis=1)
    
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('Principal Component '+str(i+1), fontsize = 15)
    ax.set_ylabel('Principal Component '+str(j+1), fontsize = 15)
    ax.set_title('2 Component PCA', fontsize = 20)
    



    targets = ['Resistant', 'Sensitive_short']
    colors = ['b', 'r']
    for target, color in zip(targets,colors):
        t=0
        if target == 'Resistant':
            t=1
        #if target == 'Sensitive_long':
         #   t=2
        indicesToKeep = finalDf[0] == t
        ax.scatter(finalDf.loc[indicesToKeep, 'principal component '+str(i)]
                   , finalDf.loc[indicesToKeep, 'principal component '+str(j)]
                   , c = color
                   , s = 50)
    ax.legend(targets)
    ax.grid()

In [ ]:
my_PCA(X, concatenated.columns, y, 0, 2)
#plt.savefig('my_PCA_res_sens_12.png')
my_PCA

In [66]:
#kf = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=36851234)
kf = StratifiedKFold(n_splits=10)
recall=[]
precision=[]
accuracy=[]
fscore=[]
scores=['true_sens', 'false_senl', 'true_senl', 'false_sens', 'accuracy', 'precision', 'recall']
performance=pd.DataFrame(index=np.arange(10), columns=scores)
i=0
for train_index, test_index in kf.split(X, y):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    '''X=pd.DataFrame(x_train, columns=features)
    model= RandomForestClassifier(n_estimators=200, max_depth=15, class_weight='balanced')
    model.fit(X,y_train)
    f=feat_importances.nlargest(15).index
    x_train=X[f].values
    X_t=pd.DataFrame(x_test, columns=features)
    x_test=X_t[f]'''
    
    print('real')
    print(y_test)
    
    clf = RandomForestClassifier(n_estimators=200, max_depth=15, class_weight='balanced')
    #clf=KNeighborsClassifier(n_neighbors=3, metric='manhattan')
    #clf = AdaBoostClassifier(n_estimators=200, base_estimator=DecisionTreeClassifier(max_depth=5),random_state=0)
    #clf=LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')
    #clf = svm.SVC(kernel='rbf', C=100, probability=True, gamma = 0.0001, class_weight='balanced', random_state=42)
    
    clf.fit(x_train, y_train)
    y_pred=clf.predict(x_test)
    print('predicted')
    print(y_pred)
    recall.append(recall_score(y_test, y_pred))
    precision.append(precision_score(y_test, y_pred))
    accuracy.append(accuracy_score(y_test, y_pred))
    performance.iloc[i]['accuracy']=accuracy_score(y_test, y_pred)
    performance.iloc[i]['precision']=precision_score(y_test, y_pred)
    performance.iloc[i]['recall']=recall_score(y_test, y_pred)
    c_matrix=confusion_matrix(y_test, y_pred)
    performance.iloc[i]['true_sens']=c_matrix[0][0]
    performance.iloc[i]['false_sens']=c_matrix[1][0]
    performance.iloc[i]['true_senl']=c_matrix[1][1]
    performance.iloc[i]['false_senl']=c_matrix[0][1]
    print (i)
    i=i+1

real
[1 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
predicted
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0
real
[1 1 1 1 0 0 0 0 0 0 0 0 0 0 0]


/home/lalli/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lalli/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


predicted
[0 0 1 1 0 0 0 0 0 0 0 0 0 0 0]
1
real
[1 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
predicted
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
2
real
[1 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
predicted
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
3
real
[1 1 1 0 0 0 0 0 0 0 0 0 0 0]


/home/lalli/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lalli/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


predicted
[0 1 0 0 0 0 0 0 0 0 0 0 0 0]
4
real
[1 1 1 0 0 0 0 0 0 0 0 0 0 0]
predicted
[1 0 1 0 0 0 0 0 0 0 0 0 0 0]
5
real
[1 1 1 0 0 0 0 0 0 0 0 0 0 0]
predicted
[0 0 0 1 0 0 0 0 0 0 0 0 0 0]
6
real
[1 1 1 0 0 0 0 0 0 0 0 0 0]
predicted
[0 0 0 0 0 0 0 0 0 0 0 0 0]
7
real
[1 1 1 0 0 0 0 0 0 0 0 0 0]


/home/lalli/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lalli/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


predicted
[1 0 0 0 0 1 0 0 0 0 0 0 0]
8
real
[1 1 1 0 0 0 0 0 0 0 0 0 0]
predicted
[1 0 0 0 0 0 0 0 0 1 0 0 0]
9


In [67]:
print('Average precision: ', np.mean(precision))
print('Average recall: ', np.mean(recall))
print('Average accuracy: ', np.mean(accuracy))
print('Standard_deviation precision: ', np.std(precision))
print('Standard_deviation recall: ', np.std(recall))
print('Standard_deviation accuracy: ', np.std(accuracy))

Average precision:  0.5
Average recall:  0.2416666666666667
Average accuracy:  0.7941025641025641
Standard_deviation precision:  0.4472135954999579
Standard_deviation recall:  0.225
Standard_deviation accuracy:  0.0655504612195348


In [68]:
performance

,true_sens,false_senl,true_senl,false_sens,accuracy,precision,recall
0,11,0,0,4,0.733333,0,0
1,11,0,2,2,0.866667,1,0.5
2,11,0,1,3,0.8,1,0.25
3,11,0,0,4,0.733333,0,0
4,11,0,1,2,0.857143,1,0.333333
5,11,0,2,1,0.928571,1,0.666667
6,10,1,0,3,0.714286,0,0
7,10,0,0,3,0.769231,0,0
8,9,1,1,2,0.769231,0.5,0.333333
9,9,1,1,2,0.769231,0.5,0.333333


In [ ]:
# ROC curve
classifier = RandomForestClassifier(n_estimators=200, max_depth=15, class_weight='balanced')
#classifier = AdaBoostClassifier(n_estimators=200, base_estimator=DecisionTreeClassifier(max_depth=5),random_state=0)
#classifier = svm.SVC(kernel='rbf', C=100, probability=True, gamma = 0.0001, class_weight='balanced', random_state=42)
#classifier=KNeighborsClassifier(n_neighbors=3, metric='manhattan')
#classifier= LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=36851234)


tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
plt.figure(figsize=(10,10))
i = 0
for train, test in cv.split(X, y):
    probas_ = classifier.fit(X[train], y[train]).predict_proba(X[test])
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3,
             label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

    i += 1
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                 label=r'$\pm$ 1 std. dev.')


plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve SenL vs SenS')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=5)
#plt.savefig('ROC_Res_vs_SenL_RF.png')

In [ ]:
k_fold = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=36851234)
y_real = []
y_proba = []
#predictor= RandomForestClassifier(n_estimators=200, max_depth=15, class_weight='balanced')
#predictor = AdaBoostClassifier(n_estimators=200, base_estimator=DecisionTreeClassifier(max_depth=5),random_state=0)
predictor = svm.SVC(kernel='rbf', C=100, probability=True, gamma = 0.0001, class_weight='balanced', random_state=42)
#predictor = KNeighborsClassifier(n_neighbors=3, metric='manhattan')
#predictor = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=36851234)


tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
plt.figure(figsize=(10,10))
i = 0
for train, test in cv.split(X, y):
    probas_ = classifier.fit(X[train], y[train]).predict_proba(X[test])
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3,
             label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

    i += 1
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                 label=r'$\pm$ 1 std. dev.')


plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
#plt.title('ROC curve Res vs Senl')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=5)
plt.savefig('ROC_Res_vs_SenS.png')
#predictor = AdaBoostClassifier(n_estimators=200, base_estimator=DecisionTreeClassifier(max_depth=5),random_state=0)
precision_array = []
threshold_array=[]
recall_array = np.linspace(0, 1, 100)
plt.figure(figsize=(10,10))
for i, (train_index, test_index) in enumerate(k_fold.split(X, y)):
    Xtrain, Xtest = X[train_index], X[test_index]
    ytrain, ytest = y[train_index], y[test_index]
    predictor.fit(Xtrain, ytrain)
    pred_proba = predictor.predict_proba(Xtest)
    precision_fold, recall_fold, thresh = precision_recall_curve(ytest, pred_proba[:,1])
    precision_fold, recall_fold, thresh = precision_fold[::-1], recall_fold[::-1], thresh[::-1]  # reverse order of results
    thresh = np.insert(thresh, 0, 1.0)
    precision_array = interp(recall_array, recall_fold, precision_fold)
    threshold_array = interp(recall_array, recall_fold, thresh)
    pr_auc = auc(recall_array, precision_array)

    lab_fold = 'Fold %d AUC=%.4f' % (i+1, pr_auc)
    plt.plot(recall_fold, precision_fold, alpha=0.3, label=lab_fold)
    y_real.append(ytest)
    y_proba.append(pred_proba[:,1])
    
y_real = np.concatenate(y_real)
y_proba = np.concatenate(y_proba)
precision, recall, thr = precision_recall_curve(y_real, y_proba)
lab = 'Overall AUC=%.4f' % (auc(recall, precision))

plt.plot(recall, precision, lw=2,color='red', label=lab)

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=5)

mean_precision = np.mean(precision_array)
std_precision = np.std(precision_array)
plt.fill_between(recall, precision + std_precision, precision - std_precision, alpha=0.3, linewidth=0, color='grey')
#plt.show()
#plt.title('PR curve Res vs SenS')
#plt.savefig('PR curve Res vs Senl_RF.png')

In [ ]:
res_senl=pd.read_csv("geni_Res_SenL.csv", delimiter=',')
res_sens=pd.read_csv("geni_Res_SenS.csv", delimiter=',')
senl_sens=pd.read_csv("geni_SenL_SenS.csv", delimiter=',')

In [ ]:
res_senl.head()

In [ ]:
lista_res_senl=pd.DataFrame(res_senl['ensemble_id'])
lista_res_sens=pd.DataFrame(res_sens['ensemble_id'])
lista_senl_sens=pd.DataFrame(senl_sens['ensemble_id'])

In [ ]:
concatenated=pd.concat([lista_res_senl, lista_res_sens, lista_senl_sens])
tutti_geni=concatenated.drop_duplicates()
tutti_geni.head()

In [ ]:
names=[]

for x in ["-".join(x.strip().split("-")[1:])  for x in (tutti_geni['ensemble_id'])]:
    names.append(x)   
    
tutti_geni['gene_names']=names
tutti_geni=pd.DataFrame(tutti_geni)
tutti_geni.to_csv('common_genes_total.csv', sep=',', header=True, index=False)